<a href="https://colab.research.google.com/github/MoqiSheng/MoqiSheng.github.io/blob/main/qwenUSPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.7 MB/s eta 0:00:00


In [3]:
import pickle
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import collections
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from sklearn import linear_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error, \
    top_k_accuracy_score, mean_squared_error, r2_score
from sklearn.model_selection import KFold
from transformers import AutoModel
from torch_geometric.nn import GATConv, GCNConv
from collections import Counter
from torch_geometric.data import Data


class SVFeatureBlock(nn.Module):
    def __init__(self, input_size=512, hidden_size=512, mode='mean'):
        super(SVFeatureBlock, self).__init__()
        self.mode = mode
        self.input_size = input_size
        self.hidden_size = hidden_size

        if mode == 'lstm':
            self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
            nn.init.orthogonal_(self.lstm.weight_ih_l0)
            nn.init.orthogonal_(self.lstm.weight_hh_l0)
        elif mode == 'bi-lstm':
            self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True,
                                bidirectional=True)
        elif self.mode == "gru":
            self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
        elif mode == 'rnn':
            self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)

    def forward(self, sv):
        sv_list = []
        for x_tmp in sv:
            if self.mode == "mean":
                if x_tmp.dim() != 1:
                    out_put = torch.mean(x_tmp, dim=0)
            elif self.mode == "sum":
                if x_tmp.dim() != 1:
                    out_put = torch.sum(x_tmp, dim=0)
            elif self.mode == "max":
                if x_tmp.dim() != 1:
                    out_put = torch.max(x_tmp, dim=0).values
            elif self.mode == "lstm":
                out_put, (h_n, c_n) = self.lstm(x_tmp.view(1, -1, self.input_size))
                out_put = out_put[:, -1, :]
                out_put = torch.squeeze(out_put)
            else:
                pass

            sv_list.append(out_put)
        x = torch.stack(sv_list)  # 拼接,(batch,512)
        return x


def weights_init_1(m):
    seed = 20
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.nn.init.xavier_uniform_(m.weight, gain=1)


def weights_init_2(m):
    seed = 20
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.nn.init.xavier_uniform_(m.weight, gain=1)
    torch.nn.init.constant_(m.bias, 0)


class Attention_Soft(nn.Module):
    def __init__(self, in_size, hidden_size=32):
        super(Attention_Soft, self).__init__()

        self.l1 = torch.nn.Linear(in_size, hidden_size, bias=True)
        self.ac = nn.Sigmoid()
        self.l2 = torch.nn.Linear(in_size, hidden_size, bias=False)
        self.l3 = torch.nn.Linear(int(hidden_size), 1, bias=False)

        weights_init_2(self.l1)
        weights_init_1(self.l2)
        weights_init_1(self.l3)

    def forward(self, z):
        w1 = self.l1(torch.mean(z, dim=1).unsqueeze(1))
        w2 = self.l2(z)
        w = self.ac(w1 + w2)
        w = self.l3(w)
        beta = torch.softmax(w, dim=1)

        return (beta * z).sum(1)


class MLP_DimReduction(nn.Module):
    def __init__(self, input_dim=4096, hidden_dim=2048, output_dim=768):
        super(MLP_DimReduction, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_dim, output_dim)
        # self.dropout = nn.Dropout(p=0.3)

        # Initialize weights
        weights_init_2(self.layer1)
        weights_init_2(self.layer2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        # x = self.dropout(x)
        x = self.layer2(x)
        return x


class SV_GAT(nn.Module):
    def __init__(self, args):
        super(SV_GAT, self).__init__()
        self.args = args

        self.length = list(np.load('/content/drive/MyDrive/USPM_16/data/length.npy'))
        pretrain_sv_path = args.pretrain_sv_path
        pretrain_scn_path = args.pretrain_scn_path
        self.sv_embedding = torch.load(pretrain_sv_path, map_location=torch.device(args.device))
        self.scn_embedding = torch.load(pretrain_scn_path, map_location=torch.device(args.device))

        # MLP for dimension reduction of scn_embedding (text) from 4096 to 768
        self.mlp_dim_reduction = MLP_DimReduction(input_dim=4096, hidden_dim=2048, output_dim=768)

        self.sv_agg = SVFeatureBlock(input_size=768, hidden_size=768, mode=args.mode)

        self.attention_soft = Attention_Soft(in_size=768)

        self.gat = GAT(input_dim=768, hidden_dim=64, output_dim=10, heads=8, args=args, drop=0.6)

        self.gat_poi = GAT_P(input_dim=768, hidden_dim=64, output_dim=4, heads=8, args=args)

    def forward(self):
        sv_features = self.sv_embedding
        street_list = list(torch.split(sv_features, self.length, dim=0))

        sv_aggre = self.sv_agg(street_list)
        sv_embedding = sv_aggre
        scn_embedding = self.mlp_dim_reduction(self.scn_embedding)  # Reduce text embedding to 768 dim

        street_embedding = self.attention_soft(torch.stack([scn_embedding, sv_embedding], dim=1))

        if self.args.downstream == 'poi':
            gat_loss, out = self.gat_poi(street_embedding)
        else:
            gat_loss, s_emb1, out = self.gat(street_embedding)

        loss = gat_loss

        return loss, out, street_embedding

    def test(self, out):
        if self.args.downstream == 'poi':
            acc, f1_score_test, mrr_test, num, pred_out = self.gat_poi.test(out)
            return acc, f1_score_test, mrr_test, 1, 1, 1, num, pred_out
        else:
            a1, a3, a5, a10, f1, mrr, num, pred_out = self.gat.test(out)
            return a1, a3, a5, a10, f1, mrr, num, pred_out


class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads, args, drop=0.6):
        super().__init__()
        torch.manual_seed(0)
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=0.6)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, concat=False,
                             heads=10, dropout=0.6)
        self.elu = nn.ELU()
        self.drop1 = nn.Dropout(p=drop)
        self.drop2 = nn.Dropout(p=0.6)

        self.edge_index = torch.load('/content/drive/MyDrive/USPM_16/data/edge_index.pt').t().contiguous().to(args.device)
        self.y = torch.from_numpy(np.load('/content/drive/MyDrive/USPM_16/data/function/label_all_function.npy', allow_pickle=True)).long().to(args.device)
        self.train_mask = torch.from_numpy(np.load('/content/drive/MyDrive/USPM_16/data/function/label_mask.npy', allow_pickle=True)).to(args.device)

        self.mask = torch.load('/content/drive/MyDrive/USPM_16/data/function/test_mask.pt')

        self.y_testlabel = np.load('/content/drive/MyDrive/USPM_16/data/function/label_all_function.npy')[self.mask]

    def forward(self, street_embedding):
        street_embedding = self.drop1(street_embedding)
        street_embedding_1 = self.conv1(street_embedding, self.edge_index)
        street_embedding_2 = self.elu(street_embedding_1)
        street_embedding_2 = self.drop2(street_embedding_2)
        street_embedding_2 = self.conv2(street_embedding_2, self.edge_index)

        cross_criterion = torch.nn.CrossEntropyLoss()
        loss_su = cross_criterion(street_embedding_2[self.train_mask], self.y[self.train_mask])

        return loss_su, street_embedding_1, street_embedding_2

    def test(self, out):
        pred = out.argmax(dim=1)
        pred = pd.DataFrame({'Type': torch.Tensor.cpu(pred).numpy()})

        predictions_test_dim = torch.Tensor.cpu(out[self.mask]).argmax(dim=1).detach().numpy()
        predictions_test = torch.Tensor.cpu(out[self.mask]).detach().numpy()

        A1 = top_k_accuracy_score(self.y_testlabel, predictions_test, k=1, labels=range(10))
        A3 = top_k_accuracy_score(self.y_testlabel, predictions_test, k=3, labels=range(10))
        A5 = top_k_accuracy_score(self.y_testlabel, predictions_test, k=5, labels=range(10))
        print(f'A1={A1}\t A3={A3}\t A5={A5} ')

        precision_score_test = precision_score(self.y_testlabel, predictions_test_dim, average="weighted")
        f1_score_test = f1_score(self.y_testlabel, predictions_test_dim, average="weighted")
        mrr_test = compute_mrr(self.y_testlabel, predictions_test)
        result = Counter(pred['Type'].values.tolist())
        num = len(result)
        print(
            f'precision={precision_score_test}, f1={f1_score_test}, mrr={mrr_test},num={num}')

        print(result)
        return A1, A3, A5, 1, f1_score_test, mrr_test, num, out


class GAT_P(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads, args, drop=0.6):
        super().__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=drop)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, concat=False,
                             heads=4, dropout=drop)
        self.elu = nn.ELU()
        self.drop1 = nn.Dropout(p=drop)
        self.drop2 = nn.Dropout(p=drop)

        self.edge_index = torch.load('/content/drive/MyDrive/USPM_16/data/edge_index.pt').t().contiguous().to(args.device)

        self.y = torch.from_numpy(np.load('/content/drive/MyDrive/USPM_16/data/poi/label_all_poi_level.npy', allow_pickle=True)).long().to(args.device)
        self.train_mask = torch.from_numpy(np.load('/content/drive/MyDrive/USPM_16/data/poi/label_mask_poi_level.npy', allow_pickle=True)).to(args.device)
        self.test_mask = torch.from_numpy(np.load('/content/drive/MyDrive/USPM_16/data/poi/test_mask_poi_level.npy', allow_pickle=True)).to(args.device)

        self.mask = torch.from_numpy(np.load('/content/drive/MyDrive/USPM_16/data/poi/test_mask_poi_level.npy', allow_pickle=True))

        self.y_testlabel = np.load('/content/drive/MyDrive/USPM_16/data/poi/label_all_poi_level.npy')[self.mask]

    def forward(self, street_embedding):
        street_embedding = self.drop1(street_embedding)
        street_embedding = self.conv1(street_embedding, self.edge_index)
        street_embedding = self.elu(street_embedding)
        street_embedding = self.drop2(street_embedding)
        street_embedding = self.conv2(street_embedding, self.edge_index)

        cross_criterion = torch.nn.CrossEntropyLoss()
        loss_su = cross_criterion(street_embedding[self.train_mask], self.y[self.train_mask])

        return loss_su, street_embedding

    def test(self, out):
        pred = out.argmax(dim=1)
        correct = pred[self.test_mask] == self.y[self.test_mask]
        acc = int(correct.sum()) / int(self.test_mask.sum())

        pred = pd.DataFrame({'Type': torch.Tensor.cpu(pred).numpy()})

        predictions_test_dim = torch.Tensor.cpu(out[self.mask]).argmax(dim=1).detach().numpy()
        predictions_test = torch.Tensor.cpu(out[self.mask]).detach().numpy()
        f1_score_test = f1_score(self.y_testlabel, predictions_test_dim, average="macro")
        mrr_test = compute_mrr(self.y_testlabel, predictions_test)
        result = Counter(pred['Type'].values.tolist())
        num = len(result)
        print(
            f'acc={acc}, f1={f1_score_test}, mrr={mrr_test},num={num}')

        print(result)
        return acc, f1_score_test, mrr_test, num, out


def compute_mrr(true_labels, machine_preds):
    """Compute the MRR """
    rr_total = 0.0
    for i in range(len(true_labels)):
        if true_labels[i] == 403:
            continue
        ranklist = list(np.argsort(machine_preds[i])[::-1])
        rank = ranklist.index(true_labels[i]) + 1
        rr_total = rr_total + 1.0 / rank
    mrr = rr_total / len(true_labels)
    return mrr

In [5]:
import itertools
import os
import random
import math
import torch
import numpy as np
import argparse
import warnings
from datetime import datetime
# from model import SV_GAT  # Assuming the modified SV_GAT is in model.py

warnings.filterwarnings('ignore')
parser = argparse.ArgumentParser()
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

parser.add_argument('--device', type=str, default='cuda:0', help='gpu device ids')
parser.add_argument('--print_num', type=int, default=2, help='gap of print evaluations')
parser.add_argument("--print_epoch", type=int, default=0, help="Start print epoch")
parser.add_argument("--start_epoch", type=int, default=0, help="Start epoch")
parser.add_argument("--current_epoch", type=int, default=0, help="Current epoch")
parser.add_argument("--epochs", type=int, default=200, help="Epochs")
parser.add_argument("--seed", type=int, default=42, help="random seed.")
parser.add_argument("--rounds", type=int, default=5, help="number of training rounds")
parser.add_argument("--mode", type=str, default='lstm', help="aggression function.")
parser.add_argument("--pretrain_scn_path", type=str, default='/content/drive/MyDrive/USPM_16/embeddings/qwen_text_embeddings_32.pt')

args = parser.parse_args([])

def trainer(args, model, optimizer1, optimizer2, optimizer3, optimizer4, epoch):
    loss_epoch = []
    model.train()
    optimizer1.zero_grad()
    optimizer2.zero_grad()
    optimizer3.zero_grad()
    optimizer4.zero_grad()

    gnn_loss, pre_out, street_embedding = model()
    loss_epoch.append(gnn_loss.item())
    loss = gnn_loss

    loss.backward()

    optimizer1.step()
    optimizer2.step()
    optimizer3.step()
    optimizer4.step()

    if epoch % args.print_num == 0:
        print(f"TrainEpoch [{epoch + 1}/{args.epochs}]\t loss_epoch_gnn:{np.mean(loss_epoch)}")
    return np.mean(loss_epoch), pre_out, street_embedding

def test(args, model, epoch, round_num, result_dir):
    with torch.no_grad():
        model.eval()
        _, out, _ = model()
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        os.makedirs(result_dir, exist_ok=True)
        if args.downstream == 'poi':
            acc, f1, mrr, _, _, _, num, pred_out = model.test(out)
            result = {
                'epoch': epoch + 1,
                'acc': acc,
                'f1': f1,
                'mrr': mrr,
                'num': num
            }
            return acc, f1, mrr, 1, 1, 1, num, pred_out, result
        else:
            a1, a3, a5, a10, f1, mrr, num, pred_out = model.test(out)
            result = {
                'epoch': epoch + 1,
                'a1': a1,
                'a3': a3,
                'a5': a5,
                'a10': a10,
                'f1': f1,
                'mrr': mrr,
                'num': num
            }
            return a1, a3, a5, a10, f1, mrr, num, pred_out, result

def calculate_best_results(result_dir, downstream):
    # Collect round files
    round_files = [f for f in os.listdir(result_dir) if f.startswith('round_') and f.endswith('.npy')]
    if not round_files:
        return None

    # Load all results and group by epoch
    epoch_results = {}
    for round_file in round_files:
        round_data = np.load(os.path.join(result_dir, round_file), allow_pickle=True).item()
        for epoch_data in round_data['epochs']:
            epoch = epoch_data['epoch']
            if epoch not in epoch_results:
                epoch_results[epoch] = []
            epoch_results[epoch].append(epoch_data)

    # Compute per-epoch averages across rounds
    epoch_averages = {}
    best_f1 = -1
    best_epoch = None

    if downstream == 'poi':
        all_results = []
        for epoch in epoch_results:
            accs = [r['acc'] for r in epoch_results[epoch]]
            f1s = [r['f1'] for r in epoch_results[epoch]]
            mrrs = [r['mrr'] for r in epoch_results[epoch]]
            epoch_averages[epoch] = {
                'avg_acc': np.mean(accs),
                'avg_f1': np.mean(f1s),
                'avg_mrr': np.mean(mrrs)
            }
            all_results.extend(epoch_results[epoch])
            if epoch_averages[epoch]['avg_f1'] > best_f1:
                best_f1 = epoch_averages[epoch]['avg_f1']
                best_epoch = epoch

        # Store best epoch results and overall max metrics
        best_results = {
            'best_epoch': best_epoch,
            'best_epoch_avg_acc': epoch_averages[best_epoch]['avg_acc'],
            'best_epoch_avg_f1': epoch_averages[best_epoch]['avg_f1'],
            'best_epoch_avg_mrr': epoch_averages[best_epoch]['avg_mrr'],
            'overall_best_acc': max([r['acc'] for r in all_results]),
            'overall_best_f1': max([r['f1'] for r in all_results]),
            'overall_best_mrr': max([r['mrr'] for r in all_results])
        }
    else:
        all_results = []
        for epoch in epoch_results:
            a1s = [r['a1'] for r in epoch_results[epoch]]
            a3s = [r['a3'] for r in epoch_results[epoch]]
            a5s = [r['a5'] for r in epoch_results[epoch]]
            a10s = [r['a10'] for r in epoch_results[epoch]]
            f1s = [r['f1'] for r in epoch_results[epoch]]
            mrrs = [r['mrr'] for r in epoch_results[epoch]]
            epoch_averages[epoch] = {
                'avg_a1': np.mean(a1s),
                'avg_a3': np.mean(a3s),
                'avg_a5': np.mean(a5s),
                'avg_a10': np.mean(a10s),
                'avg_f1': np.mean(f1s),
                'avg_mrr': np.mean(mrrs)
            }
            all_results.extend(epoch_results[epoch])
            if epoch_averages[epoch]['avg_f1'] > best_f1:
                best_f1 = epoch_averages[epoch]['avg_f1']
                best_epoch = epoch

        # Store best epoch results and overall max metrics
        best_results = {
            'best_epoch': best_epoch,
            'best_epoch_avg_a1': epoch_averages[best_epoch]['avg_a1'],
            'best_epoch_avg_a3': epoch_averages[best_epoch]['avg_a3'],
            'best_epoch_avg_a5': epoch_averages[best_epoch]['avg_a5'],
            'best_epoch_avg_a10': epoch_averages[best_epoch]['avg_a10'],
            'best_epoch_avg_f1': epoch_averages[best_epoch]['avg_f1'],
            'best_epoch_avg_mrr': epoch_averages[best_epoch]['avg_mrr'],
            'overall_best_a1': max([r['a1'] for r in all_results]),
            'overall_best_a3': max([r['a3'] for r in all_results]),
            'overall_best_a5': max([r['a5'] for r in all_results]),
            'overall_best_a10': max([r['a10'] for r in all_results]),
            'overall_best_f1': max([r['f1'] for r in all_results]),
            'overall_best_mrr': max([r['mrr'] for r in all_results])
        }

    # Save per-epoch averages and best results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    np.save(f'{result_dir}/epoch_averages_{timestamp}.npy', epoch_averages)
    np.save(f'{result_dir}/best_results_{timestamp}.npy', best_results)
    return best_results

def run_training(pretrain_sv_path, result_subdir):
    result_dir = f'/content/drive/MyDrive/USPM_16/result/{result_subdir}'
    os.makedirs(result_dir, exist_ok=True)

    for downstream in ['function', 'poi']:
        print(f"\nStarting {downstream} downstream with {pretrain_sv_path}")
        args.downstream = downstream
        args.pretrain_sv_path = pretrain_sv_path
        args.current_epoch = 0

        for round_num in range(args.rounds):
            print(f"Round {round_num + 1}/{args.rounds}")

            np.random.seed(args.seed + round_num)
            random.seed(args.seed + round_num + 1)
            torch.manual_seed(args.seed + round_num + 2)
            torch.cuda.manual_seed(args.seed + round_num + 3)
            torch.backends.cudnn.deterministic = True

            model = SV_GAT(args)
            model = model.to(args.device)

            opt1 = torch.optim.Adam(
                itertools.chain(model.attention_soft.parameters()),
                lr=0.0005, weight_decay=1e-8)
            opt4 = torch.optim.Adam(
                model.mlp_dim_reduction.parameters(),
                lr=0.0005, weight_decay=1e-8)  # Optimizer for MLP dimension reduction
            if args.downstream == 'poi':
                opt3 = torch.optim.Adam(model.gat_poi.parameters(), lr=0.0005, weight_decay=5e-4)
                args.epochs = 250
            else:
                opt3 = torch.optim.Adam(model.gat.parameters(), lr=0.005, weight_decay=5e-4)
                args.epochs = 200

            if args.mode != 'mean':
                opt2 = torch.optim.SGD(model.sv_agg.parameters(), lr=0.005, weight_decay=1e-4, momentum=0.9)
                t = 10
                T = 800
                n_t = 0.5
                lf = lambda epoch: (0.9 * epoch / t + 0.1) if epoch < t else 0.1 if n_t * (
                        1 + math.cos(math.pi * (epoch - t) / (T - t))) < 0.1 else n_t * (
                        1 + math.cos(math.pi * (epoch - t) / (T - t)))
                scheduler = torch.optim.lr_scheduler.LambdaLR(opt2, lr_lambda=lf)
            else:
                opt2 = torch.optim.SGD(model.sv_agg.parameters(), lr=0.005, weight_decay=1e-4, momentum=0.9)

            print(model)

            # Collect results for this round
            round_results = {'round': round_num, 'epochs': []}

            for epoch in range(args.start_epoch, args.epochs):
                loss_epoch, pred_, street_embedding = trainer(args, model, opt1, opt2, opt3, opt4, epoch)
                if args.mode != 'mean':
                    scheduler.step()
                if epoch % args.print_num == 0:
                    result_tuple = test(args, model, epoch, round_num, f'{result_dir}/{downstream}')
                    # Append result to round_results
                    round_results['epochs'].append(result_tuple[-1])  # Last element is the result dict

            # Save all results for this round in a single file
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            np.save(f'{result_dir}/{downstream}/round_{round_num}_{timestamp}.npy', round_results)

        # Calculate and save average and best results
        best_results = calculate_best_results(f'{result_dir}/{downstream}', downstream)
        print(f"Best Results for {downstream}:", best_results)

run_training('/content/drive/MyDrive/USPM_16/embeddings/image_representation_117144_16.pt', 'image_16')


Starting function downstream with /content/drive/MyDrive/USPM_16/embeddings/image_representation_117144_16.pt
Round 1/5
SV_GAT(
  (mlp_dim_reduction): MLP_DimReduction(
    (layer1): Linear(in_features=4096, out_features=2048, bias=True)
    (relu): ReLU()
    (layer2): Linear(in_features=2048, out_features=768, bias=True)
  )
  (sv_agg): SVFeatureBlock(
    (lstm): LSTM(768, 768, batch_first=True)
  )
  (attention_soft): Attention_Soft(
    (l1): Linear(in_features=768, out_features=32, bias=True)
    (ac): Sigmoid()
    (l2): Linear(in_features=768, out_features=32, bias=False)
    (l3): Linear(in_features=32, out_features=1, bias=False)
  )
  (gat): GAT(
    (conv1): GATConv(768, 64, heads=8)
    (conv2): GATConv(512, 10, heads=10)
    (elu): ELU(alpha=1.0)
    (drop1): Dropout(p=0.6, inplace=False)
    (drop2): Dropout(p=0.6, inplace=False)
  )
  (gat_poi): GAT_P(
    (conv1): GATConv(768, 64, heads=8)
    (conv2): GATConv(512, 4, heads=4)
    (elu): ELU(alpha=1.0)
    (drop1): Dr